In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import anndata as ad
import scanpy as sc
import seaborn as sns

np.random.seed(84)

In [2]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

out_data_path = "../data/"

scanpy==1.9.3 anndata==0.9.1 umap==0.5.3 numpy==1.23.5 scipy==1.10.0 pandas==1.5.3 scikit-learn==1.2.1 statsmodels==0.13.5 python-igraph==0.10.5 pynndescent==0.5.10


### Load the Data

In [3]:
cells_data_path = out_data_path + "cells_ad.h5ad"
adata = ad.read_h5ad(cells_data_path)

In [4]:
adata.uns['log1p']["base"] = None

In [5]:
adata

AnnData object with n_obs × n_vars = 20248 × 30267
    obs: 'dataset', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'doublet_score', 'predicted_doublet', 'S_score', 'G2M_score', 'phase', 'leiden', 'HSC_expr', 'HSC_cell', 'HSPC_expr', 'HSPC_cell', 'Myeloid_expr', 'Myeloid_cell', 'Erythroid_expr', 'Erythroid_cell', 'MK_expr', 'MK_cell', 'MAST_expr', 'MAST_cell', 'Lymphoid_expr', 'Lymphoid_cell'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'mean', 'std'
    uns: 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'phase_colors', 'scrublet', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

## Differential Gene Expression
### t-test

In [6]:
celltype_markers = {'HSC':['PROM1','AVP','HLF'],
                    'HSPC':['PROM1','CD34'],
                    'Myeloid':['MPO', 'S100A8', 'S100A9', 'ELANE', 'AZU1'],
                    'Erythroid':['KLF1', 'GATA1', 'TFRC'],
                    'MK':['PF4', 'ITGA2B', 'GP9'],
                    'MAST':['KIT', 'KRT1'],
                    'Lymphoid':['IL7R', 'CD7', 'MME']}

In [7]:
for cell_type in celltype_markers:
    
    print(cell_type)
    # Set up the string the cell column
    cell_col = cell_type + '_cell'
    
    # Get the cells in the column
    cells = adata[adata.obs[cell_col] == cell_type]
    
    # Perform the differential expression analysis using ttest method. Use the raw data set in 2_dataprocessing.
    sc.tl.rank_genes_groups(cells, 'dataset', groups=['SF3B1'], reference='AAVS', method='t-test', pts=True, use_raw=True)
    
    # Extract DE genes information
    de_results = cells.uns['rank_genes_groups']
    de_df = pd.DataFrame({
        'genes': de_results['names']['SF3B1'],
        'logfoldchanges': de_results['logfoldchanges']['SF3B1'],
        'pvals': de_results['pvals']['SF3B1'],
        'pvals_adj': de_results['pvals_adj']['SF3B1'],
    })

    # Save DataFrame to CSV
    unfltd_path = "../deg_output/ttest/unfiltered_" + cell_type + ".csv"
    de_df.to_csv(unfltd_path, index=False)
    
    # Save the gene content in cells
    cells_pts = de_results['pts']
    pts_path = "../deg_output/ttest/pts_" + cell_type + ".csv"
    cells_pts.to_csv(pts_path)
    
    del(cells, de_results, de_df, cells_pts)

HSC
ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)


/Users/nellia/anaconda3/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:580: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


HSPC
ranking genes


/Users/nellia/anaconda3/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:580: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)
Myeloid
ranking genes


...

/Users/nellia/anaconda3/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:580: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


ranking genes


/Users/nellia/anaconda3/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:580: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}


    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:01)
Lymphoid
ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)


/Users/nellia/anaconda3/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:580: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
